## Callbacks, hooks & introspection (Chap 15)

#### Callbacks & hooks
- The use of callbacks and hooks is a common meta-programming technique. They are called when an event occur during Ruby program execution.

In [4]:
# intercepting unknown messages with method_missing
# (arguably the most commonly used runtime hook in Ruby)
# delegation

class Cookbook
    attr_accessor :title, :author
    def initialize
        @recipes=[]
    end
    def method_missing(m,*args,&block)
        @recipes.public_send(m,*args,&block)
    end
end

class Recipe
    attr_accessor :main_ingredient
    def initialize(main_ingredient)
        @main_ingredient=main_ingredient
    end
end

cb = Cookbook.new
recipe_for_cake = Recipe.new("flour")
recipe_for_chicken = Recipe.new("chicken")

<<README
cb doesn’t have methods called << and select , so those messages are 
delegated to @recipes via method_missing.
README

cb << recipe_for_cake; cb << recipe_for_chicken

chicken_dishes = cb.select {|recipe| recipe.main_ingredient == "chicken"}
chicken_dishes.each {|dish| puts dish.main_ingredient}

chicken


[#<Recipe:0x000056025fff8770 @main_ingredient="chicken">]

In [5]:
# often-cited problem: method_missing doesn't correspond to respond_to.
class Person
    attr_accessor :name, :age
    def initialize(name,age)
        @name,@age=name,age
    end
    
    # in this example, intercept messages starting with "set_"
    # and transform them into setter methods.
    
    def method_missing(m,*args,&block)
        if /set_(.*)/.match(m)
            self.public_send("#{$1}=",*args)
        else
            super
        end
    end
    
    # get method_missing & respond_to? to line up with each other 
    # by defining respond_to_missing?    
    def respond_to_missing?(m,include_private=false)
        /set_/.match(m) || super 
    end
    
end

p = Person.new("joe",37)
p.set_age(38); print p.age
print p.respond_to?(:set_age)

38true

In [6]:
# trapping include & prepend operations
# with "included" and "prepended"

module M
    def self.included(c)
        puts "just mixed into #{c}."
    end
end
class C
    include M
end

just mixed into C.


C

In [8]:
# When should a module intercept its own inclusion? One common case revolves 
# the difference between instance and class methods. 
# When you mix a module into a class, you’re ensuring that all the instance
# methods defined in the module become available to instances of the class. 
# But the class object isn’t affected, so what if you want to add class 
# methods to the class by mixing in the module along with adding the 
# instance methods?
# Courtesy of included, you can trap the include operation and use it to
# add class methods to the class that’s doing the including.

module M
    def self.included(cl)
        def cl.a_class_method
            puts "Now the class has a new class method."
        end
    end
    def an_inst_method
        puts "This module supplies this instance method."
    end
end

class C
    include M
end

# When class C includes module M , two things happen. 
# 1) an instance method called an_inst_method appears in the lookup path of 
# its instances (such as c ). 
# 2) thanks to M ’s included callback, a class method called a_class_method 
# is defined for C .
C.a_class_method
c = C.new
c.an_inst_method

Now the class has a new class method.
This module supplies this instance method.


In [9]:
# intercepting extend
# extending individual objects with modules is one of Ruby's most powerful 
# techniques for customizing objects. 
# It’s also the beneficiary of a runtime hook: using Module#extend,
# you can set up a callback that will be triggered whenever an object 
# performs an extend operation involving the module.

module M
    def self.extended(obj)
        puts "Module #{self} is being used by #{obj}."
    end
    def an_inst_method
        puts "This module supplies this instance method."
    end
end
my_object = Object.new
my_object.extend(M)
my_object.an_inst_method

Module M is being used by #<Object:0x00005617257b4fc0>.
This module supplies this instance method.


In [10]:
# singleton-class behavior
# extending an object with a module is the same as including that module in
# the object’s singleton class.
# The module is added to the object’s method-lookup path, entering the 
# chain right after the object’s singleton class.
# But the two operations trigger different callbacks: extended and included .

module M
    def self.included(c)
        puts "#{self} included by #{c}."
    end
    def self.extended(obj)
        puts "#{self} extended by #{obj}."
    end
end

obj = Object.new
puts "Including M in object's singleton class:"

class << obj
    include M
end

obj = Object.new
puts "Extending object with M:"
obj.extend(M)

Including M in object's singleton class:
M included by #<Class:#<Object:0x00005617255d7220>>.
Extending object with M:
M extended by #<Object:0x00005617255c59a8>.


#<Object:0x00005617255c59a8>

In [11]:
# intercepting inheritance with Class#inherited
# You can hook into the subclassing of a class by defining a special class 
# method called inherited for that class. 
# If inherited is defined for a given class, then when you subclass the class, 
# inherited is called with the name of the new class as its single argument.

class C
    def self.inherited(subclass)
        puts "#{self} just got subclassed by #{subclass}."
    end
end
class D < C
end

C just got subclassed by D.


In [1]:
# Module#const_missing
# used whenever an unknown constant is referred to inside a module or class.

class C
    def self.const_missing(const)
        puts "#{const} is undefined—setting it to 1."
        const_set(const,1)
    end
end
puts C::A
puts C::A # 2nd call: A already defined, so const_missing isn't needed.

A is undefined—setting it to 1.
1
1


In [2]:
# method_added 
# if you define "method_added" as a class method, it's called when any
# instance method is defined or redefined.
class C
    def self.method_added(m)
        puts "Method #{m} was just defined."
    end
    def a_new_method
    end
end

Method a_new_method was just defined.


:a_new_method

In [3]:
# singleton_method_added
# same as "method_added", but for singletons.
class C
    def self.singleton_method_added(m)
        puts "Method #{m} was just defined."
    end
end

Method singleton_method_added was just defined.


:singleton_method_added

In [4]:
# singleton_method_added - also triggered by defining another singleton
# (class) method.
class C
    def self.singleton_method_added(m)
        puts "Method #{m} was just defined."
    end
    def self.new_class_method
    end
end

Method singleton_method_added was just defined.
Method new_class_method was just defined.


:new_class_method

In [5]:
# usually you should use singleton_method_added with objects other than
# class objects.
obj = Object.new
def obj.singleton_method_added(m)
    puts "Singleton method #{m} was just defined."
end
def obj.a_new_singleton_method
end

Singleton method singleton_method_added was just defined.
Singleton method a_new_singleton_method was just defined.


:a_new_singleton_method

In [6]:
# combining class- & object-based techniques
# to get object-specific effects ty defining relevant methods
# in object's singleton class:

obj = Object.new
class << obj
    def singleton_method_added(m)
        puts "Singleton method #{m} was just defined."
    end
    def a_new_singleton_method
    end
end

Singleton method singleton_method_added was just defined.
Singleton method a_new_singleton_method was just defined.


:a_new_singleton_method

In [7]:
# defining singleton_method_added as a regular instance method of a class
# - every instance of that class will trigger a callback with the creation
# of a singleton method:

class C
    def singleton_method_added(m)
        puts "Singleton method #{m} was just defined."
    end
end
c = C.new
def c.a_singleton_method
C
end

Method singleton_method_added was just defined.
Singleton method a_singleton_method was just defined.


:a_singleton_method

#### interpreting object capability queries

In [8]:
# non-private (public or protected) methods
s = "test string"

# grep filters any symbol that doesn't contain "case".
s.methods.grep(/case/).sort

[:casecmp, :casecmp?, :downcase, :downcase!, :swapcase, :swapcase!, :upcase, :upcase!]

In [10]:
# looking for bang (!) methods
s.methods.grep(/.!/).sort

[:capitalize!, :chomp!, :chop!, :delete!, :delete_prefix!, :delete_suffix!, :downcase!, :encode!, :gsub!, :lstrip!, :next!, :reverse!, :rstrip!, :scrub!, :slice!, :squeeze!, :strip!, :sub!, :succ!, :swapcase!, :tr!, :tr_s!, :unicode_normalize!, :upcase!]

In [13]:
# look for string bang methods that don't have corresponding non-bang methods
s = "test string"
def s.surprise!; end
m = s.methods
bangs = s.methods.grep(/.!/)
unmatched = bangs.reject do |b|
    m.include?(b[0..-2].to_sym)
end
if unmatched.empty?
    puts "all bang methods have non-bang matches"
else
    puts "here's some bang methods without non-bang matches"
    puts unmatched
end

here's some bang methods without non-bang matches
[:surprise!]


In [15]:
# private & protected methods
# new objects have several private methods, no protected methods.
o = Object.new
puts o.private_methods.size
puts o.protected_methods.size

80
0


In [17]:
# those private methods are mostly defined in Kernel (module) and
# BasicObject (class).
puts BasicObject.private_instance_methods(false)
puts      Kernel.private_instance_methods(false)

[:initialize, :method_missing, :singleton_method_added, :singleton_method_removed, :singleton_method_undefined]
[:sprintf, :format, :Integer, :Float, :String, :Array, :Hash, :local_variables, :rand, :srand, :readlines, :p, :trap, :warn, :system, :raise, :fail, :global_variables, :__method__, :__callee__, :__dir__, :require, :require_relative, :autoload, :autoload?, :binding, :eval, :iterator?, :block_given?, :catch, :throw, :loop, :exit!, :abort, :exec, :JSON, :spawn, :URI, :trace_var, :untrace_var, :at_exit, :gem, :`, :select, :j, :Rational, :Complex, :respond_to_missing?, :caller_locations, :caller, :test, :Pathname, :gets, :proc, :lambda, :fork, :initialize_copy, :initialize_clone, :initialize_dup, :exit, :set_trace_func, :jj, :sleep, :gem_original_require, :pp, :load, :syscall, :open, :printf, :print, :putc, :puts, :readline]


In [19]:
# class & module instance methods
String.methods.grep(/methods/).sort

[:instance_methods, :methods, :private_instance_methods, :private_methods, :protected_instance_methods, :protected_methods, :public_instance_methods, :public_methods, :singleton_methods]

In [21]:
# when calling these methods with an argument:
# False = returns methods defined in the class/module being queried
# Boolean truth = returns (above) plus ancestor classes/modules
puts Range.instance_methods(false).sort
puts
puts Range.instance_methods(false) & Enumerable.instance_methods(false)

[:%, :==, :===, :begin, :bsearch, :count, :cover?, :each, :end, :entries, :eql?, :exclude_end?, :first, :hash, :include?, :inspect, :last, :max, :member?, :min, :minmax, :pretty_print, :size, :step, :to_a, :to_s]

[:to_a, :entries, :count, :include?, :max, :min, :first, :minmax, :member?]


In [24]:
# object singleton methods
# (recall: singleton methods are defined for sole use of a particular object)
class C
end
c = C.new
class << c
    def x; end
    def y; end
    def z; end
    protected :y
    private :z
end
puts c.methods.sort
puts c.singleton_methods.sort

# z not returned, b/c singleton_methods doesn't return private methods.

Singleton method x was just defined.
Singleton method y was just defined.
Singleton method z was just defined.
[:!, :!=, :!~, :<=>, :==, :===, :=~, :__binding__, :__id__, :__send__, :a_new_method, :class, :clone, :define_singleton_method, :display, :dup, :enum_for, :eql?, :equal?, :extend, :freeze, :frozen?, :hash, :inspect, :instance_eval, :instance_exec, :instance_of?, :instance_variable_defined?, :instance_variable_get, :instance_variable_set, :instance_variables, :is_a?, :itself, :kind_of?, :method, :methods, :nil?, :object_id, :pretty_inspect, :pretty_print, :pretty_print_cycle, :pretty_print_inspect, :pretty_print_instance_variables, :private_methods, :protected_methods, :pry, :public_method, :public_methods, :public_send, :remove_instance_variable, :respond_to?, :send, :singleton_class, :singleton_method, :singleton_method_added, :singleton_methods, :taint, :tainted?, :tap, :then, :to_enum, :to_json, :to_s, :trust, :untaint, :untrust, :untrusted?, :x, :y, :yield_self]
[:x, :y]


In [25]:
# find class methods File inherits from ancestors (not defined by File)
puts File.singleton_methods - File.singleton_methods(false)

[:readlines, :new, :select, :write, :read, :sysopen, :for_fd, :popen, :foreach, :binread, :binwrite, :pipe, :copy_stream, :try_convert, :open]


#### introspection: variables & constants

In [27]:
# local & global variables
x=1
puts local_variables
puts global_variables.sort

[:_i24, :_24, :x, :_i23, :_23, :_i22, :_22, :_i21, :_21, :_i20, :_20, :overrides, :enum_classes, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :_i13, :_13, :_i12, :_12, :o, :_i11, :_11, :_i10, :_10, :m, :bangs, :unmatched, :_i9, :_9, :_i8, :_8, :s, :_i7, :_7, :c, :_i6, :_6, :_i5, :_5, :obj, :_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :pry_instance, :_out_, :_in_, :_oh, :_ih, :version, :str]
[:$!, :$", :$$, :$&, :$', :$*, :$+, :$,, :$-0, :$-F, :$-I, :$-K, :$-W, :$-a, :$-d, :$-i, :$-l, :$-p, :$-v, :$-w, :$., :$/, :$0, :$:, :$;, :$<, :$=, :$>, :$?, :$@, :$ARGV, :$CHILD_STATUS, :$CODERAY_DEBUG, :$DEBUG, :$DEFAULT_INPUT, :$DEFAULT_OUTPUT, :$ERROR_INFO, :$ERROR_POSITION, :$FIELD_SEPARATOR, :$FILENAME, :$FS, :$IGNORECASE, :$INPUT_LINE_NUMBER, :$INPUT_RECORD_SEPARATOR, :$KCODE, :$LAST_MATCH_INFO, :$LAST_PAREN_MATCH, :$LAST_READ_LINE, :$LOADED_FEATURES, :$LOAD_PATH, :$MATCH, :$NR, :$OFS, :$ORS, :$OUTPUT_FIE

In [28]:
# instance variables
class Person
    attr_accessor :name,:age
    def initialize(name)
        @name=name
    end
end
joe = Person.new("joe"); puts joe.instance_variables
joe.age = 37;            puts joe.instance_variables

[:@name]
[:@name, :@age]


#### tracing execution

In [47]:
# caller - provides an array of strings. 
# Each string represents one step in the stack trace: 
# The strings contain information about the file or program where the method
# call was made, the line where the method call occurred, and the method from
#which the current method was called, if any.

load "stacktrace.rb"

stacktrace: 
["stacktrace.rb:5:in `y'", "stacktrace.rb:2:in `x'", "stacktrace.rb:11:in `<top (required)>'", "(pry):352:in `load'", "(pry):352:in `<main>'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:290:in `eval'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:290:in `evaluate_ruby'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:659:in `handle_line'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:261:in `block (2 levels) in eval'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:260:in `catch'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:260:in `block in eval'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:259:in `catch'", "/usr/local/lib/ruby/gems/2.7.0/gems/pry-0.13.1/lib/pry/pry_instance.rb:259:in `eval'", "/home/bjpcjp/.gem/ruby/2.7.0/gems/iruby-0.4.0/lib/iruby/backend.rb:66:in `eval'", "/home/bjp

true

In [48]:
# writing a stack trace parsing tool

# Given a stack trace, generate an array of objects, each of which has 
# knowledge of a program or filename, a line number, and a method name
# (or <main> ).

module CallerTools
    class Call
        CALL_RE = /(.*):(\d+):in `(.*)'/
        attr_reader :program, :line, :meth
        def initialize(string)
            @program,@line,@meth = CALL_RE.match(string).captures
        end
        def to_s
            "%30s%5s%15s" % [program, line, meth]
        end
    end
    
    class Stack
        def initialize
            stack = caller
            stack.shift
            @backtrace = stack.map do |call|
                Call.new(call)
            end
        end
        def report
            @backtrace.map do |call|
                call.to_s
            end
        end
        def find(&block)
            @backtrace.find(&block)
        end
    end
end

(pry):361: warning: already initialized constant CallerTools::Call::CALL_RE
(pry):310: warning: previous definition of CALL_RE was here


:find

In [4]:
%x("callertest.rb")

""

#### callbacks & method inspection in practice

In [50]:
# minitest -- Ruby std testing framework

module PlayingCards
    RANKS = %w{ 2 3 4 5 6 7 8 9 10 J Q K A }
    SUITS = %w{ clubs diamonds hearts spades }
    class Deck
        def initialize
            @cards = []
            RANKS.each do |r|
                SUITS.each do |s|
                    @cards << "#{r} of #{s}"
                end
            end
            @cards.shuffle!
        end
        def deal(n=1)
            @cards.pop(n)
        end
        def size
            @cards.size
        end
    end
end

:size

In [3]:
# NOT YET RUNNING 
require 'minitest/autorun'
#require_relative "cards"
class CardTest < MiniTest::Test
    def setup
        @deck = PlayingCards::Deck.new
    end
    def test_deal_one
        @deck.deal
        assert_equal(51,@deck.size)
    end
    def test_deal_many
        @deck.deal(5)
        assert_equal(47,@deck.size)
    end
end

:test_deal_many

In [ ]:
# minitest - specifying & implementing